# Finding brain regions and getting their center of mass using the Allen SDK

1. Create a new python virtual environment or use an existing one.
1. pip install allensdk==2.15.1 # this works with the current ubuntu/pipeline we have, you can try just doing pip install allensdk
1. The Allen data has to be download which happens in the commands below. You might get an error on downloading, if you do, just copy the link that is in the error message and use curl or wget or just put that link in the browser. It will then download. You will need to move that into the current working directory of this notebook under the 'mouse_connectivity' directory


In [1]:
import numpy as np
import pandas as pd
from scipy.ndimage import center_of_mass
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from matplotlib import pyplot as plt

### Accessing the CCF from the MouseConnectivityCache

Instantiate the MouseConnectivityCache (mcc). The default resolution is 25 microns, but 10,50, and 100 microns are also valid options. You can also specify a location of a manifest file, which would keep track of all downloaded assets from template and annotation volumes to experimental data. If a manifest file directory is not provided, one will be created in the current working directory. (This is where you might get downloading errors, just download it manually and place it under the dir: mouse_connectivity'

In [ ]:
mcc = MouseConnectivityCache(resolution=25)

#### Annotated Volume and Reference Space
Now you can get the Reference Space (rsp) at that resolution. This automatically downloads the annotated volume at the specified resolution (25 microns).

In [ ]:
rsp = mcc.get_reference_space()

The annotated volume is a 3D numpy ndarray, with axes correspond to AP,DV, and ML respectively:

In [ ]:
print('Type of annotated volume', type(rsp))
print('Shape of annotated volume:',rsp.annotation.shape)
print('Annotated volume data type:',rsp.annotation.dtype)

The rsp provides access to the structure tree from the ontology. Given that there are multiple atlases and associated ontologies, we are only interested in the structures belonging to this annotation. Thus, we remove unassigned structures from the structure tree. This returns a structure graph which is more easily viewed as a pandas DataFrame:

In [ ]:
sg = rsp.remove_unassigned()

In [ ]:
sg = pd.DataFrame(sg)

In [ ]:
sg.head(10)

In [ ]:
# I picked the cerebellum as it is easy to see
# get data with structure name
structure = 'facial nerve'
row = sg.loc[sg['name'] == structure]
row

In [ ]:
# get data with acronym
# our facial nerve is 7n_L and 7n_R, Allen has it as Vlln
acronym = 'VIIn'
row = sg.loc[sg['acronym'] == acronym]
row

In [ ]:
sg.query('name.str.contains("facial")', engine='python')

In [ ]:
structure_id = row.iat[0, 3]

In [ ]:
# Get a list of acronymns
id_acronym_map = rsp.structure_tree.get_id_acronym_map() # Dictionary returns IDs given acronyms
acronym_id_map = {v:k for k,v in id_acronym_map.items()} # Flip key:value pairs to get dictionary for acronyms given IDs
print(list(map(acronym_id_map.get,rsp.structure_tree.descendant_ids([structure_id])[0])))

The structures' acronyms, names and IDs can all be linked through the structure graph.

While the rsp can be used to generate sections in the coronal (0), horizontal (1), or sagittal (2) planes:

In [ ]:
orientation = {'coronal':0,'horizontal':1,'sagittal':2} # makes orientation arguments more readible
pos_microns = lambda x:x*25 # get_slice_image function takes positional arguments in microns instead of 25 micron increments

img = rsp.get_slice_image(orientation['sagittal'],pos_microns(200))
plt.imshow(img)

The rsp can also generate masks of given structure IDs. The IDs are provided as a list, and by default include all voxels annotated as children of the given structures (direct_only=False). If only the parent level voxels are desired, set direct_only to True.

Here we make a mask of the Cerebral Cortex, which from the structure graph is shown to have the ID 688.

In [ ]:
structure_mask = rsp.make_structure_mask([structure_id], direct_only=False)

In [ ]:
print('Shape of entire brain', rsp.annotation.shape)
print('Shape of mask:',structure_mask.shape)
print('Mask data type:',structure_mask.dtype)
values, counts = np.unique(structure_mask, return_counts=True)
com = center_of_mass(structure_mask)
print(f'Center of mass for {structure}: {com}')

In [ ]:
com_section = int(round(com[2])) 
plt.scatter([com[1]], [com[0]], c='r', s=40)
plt.imshow(structure_mask[:,:,com_section])

In [55]:
def create_com(structure_id):
    com = (0,0,0)
    try:
        structure_mask = rsp.make_structure_mask([structure_id], direct_only=False)
    except:
        structure_mask = None
    if structure_mask is not None:
        com = center_of_mass(structure_mask)
    return com
def round_com(com):
    x,y,z = com
    return (round(x,2), round(y,2), round(z,2))
def calculate_distance(com1, com2):
    a = np.array([float(i) for i in eval(com1)]).astype(np.float64)
    b = np.array([float(i) for i in eval(com2)]).astype(np.float64)
    return (np.linalg.norm(a-b))*25
    
com = create_com(526157192)
print(com)
com_round = round_com(com)
print(com_round)

(0, 0, 0)
(0, 0, 0)


In [30]:
x1 = 5
y1 = 2
z1 = 4

x2 = 0
y2 = 0
z2 = 0
x = (x1-x2)**2
y = (y1-y2)**2
z = (z1-z2)**2
distance = (x+y+z)**0.5
print(distance)

6.708203932499369


In [33]:
com1 = (368.09, 62.22, 227.68)
com2 = (369.09,64.03,219.32)
a = np.array(com1)
b = np.array(com2)
np.linalg.norm(a-b)

8.611950998467202

In [35]:
calculate_distance(com1, com2)

(368.09, 62.22, 227.68) (369.09, 64.03, 219.32)


8.611950998467202

In [ ]:
#sg['com'] = create_com(sg['id'])
sg['com'] = sg['id'].apply(create_com)

In [ ]:
sg['com_rounded'] = sg['com'].apply(round_com)

In [ ]:
df = sg.drop(sg.columns[[1, 2, 5,6,7,8]], axis=1)

In [ ]:
sg.head()

In [ ]:
df.head()

In [58]:
outfile = '/home/eddyod/allen-coms.csv'
com_df.to_csv(outfile, index=False)

In [5]:
infile = '/home/eddyod/sql/allen.DK52.COMS.csv'
com_df = pd.read_csv(infile)
com_df.head()

,acronym,ID,Allen Terminology,allen_com,UCSD Terminology,dk52_l_com,dk52_r_com
0,SCs,302,"Superior colliculus, sensory related","(368.09, 62.22, 227.68)",SC,"(369.09,64.03,219.32)","(369.09,64.03,219.32)"
1,ICc,811,"Inferior colliculus, central nucleus","(416.1, 93.29, 227.68)",IC,"(416.77,77.57,225.96)","(416.77,77.57,225.96)"
2,PBG,874,Parabigeminal nucleus,"(376.07, 153.81, 227.21)",PBG,"(369.77,146.82,145.24)","(372.49,154.53,308.30)"
3,SNr,381,"Substantia nigra, reticular part","(337.62, 206.69, 227.71)",SNR,"(339.93,195.32,153.43)","(338.77,202.11,290.72)"
4,III,35,Oculomotor nucleus,"(364.12, 151.77, 227.75)",3N,"(358.99,155.14,222.43)","(356.17,154.13,233.12)"


In [56]:
#com_df['distance_l'] = com_df[['allen_com', 'dk52_l_com']].apply(calculate_distance)
com_df['distance_l'] = com_df.apply(lambda x: calculate_distance(x.allen_com, x.dk52_l_com), axis=1)

In [57]:
com_df.head(20)

,acronym,ID,Allen Terminology,allen_com,UCSD Terminology,dk52_l_com,dk52_r_com,distance_l
0,SCs,302,"Superior colliculus, sensory related","(368.09, 62.22, 227.68)",SC,"(369.09,64.03,219.32)","(369.09,64.03,219.32)",215.298775
1,ICc,811,"Inferior colliculus, central nucleus","(416.1, 93.29, 227.68)",IC,"(416.77,77.57,225.96)","(416.77,77.57,225.96)",395.700092
2,PBG,874,Parabigeminal nucleus,"(376.07, 153.81, 227.21)",PBG,"(369.77,146.82,145.24)","(372.49,154.53,308.30)",2062.709232
3,SNr,381,"Substantia nigra, reticular part","(337.62, 206.69, 227.71)",SNR,"(339.93,195.32,153.43)","(338.77,202.11,290.72)",1879.516460
4,III,35,Oculomotor nucleus,"(364.12, 151.77, 227.75)",3N,"(358.99,155.14,222.43)","(356.17,154.13,233.12)",203.064337
5,IV,115,Trochlear nucleus,"(383.51, 151.05, 227.98)",4N,"(378.42,155.75,214.87)","(377.89,157.00,237.17)",370.700519
6,SNc,374,"Substantia nigra, compact part","(334.2, 204.95, 227.85)",SNC,"(316.13,206.56,169.94)","(329.01,196.54,290.55)",1517.128270
7,P,771,Pons,"(394.79, 221.28, 227.7)",Pn,"(346.57,253.20,204.73)","(350.02,253.64,255.41)",1555.569771
8,NLL,612,Nucleus of the lateral lemniscus,"(378.59, 206.99, 227.76)",VLL,"(367.29,211.98,154.19)","(372.64,215.14,283.04)",1864.995811
9,V,621,Motor nucleus of trigeminal,"(407.73, 211.44, 227.62)",5N,"(402.39,206.28,172.58)","(400.76,209.76,281.29)",1388.466510
